# Data processing for FlexSUS

institution: DTU

author: [tilseb](mailto:tilseb@dtu.dk)

date created: 2020-01-30

licensed under: [GNU GENERAL PUBLIC LICENSE Version 3, 29 June 2007](https://www.gnu.org/licenses/gpl-3.0.html)

## Installation

1. Installation via miniconda. Get miniconda with Python 3.7 [here](https://docs.conda.io/en/latest/miniconda.html).
2. Open the anaconda promt: press `start button` , type `anaconda promt` and hit enter.
3. Navigate to the folder containing this script: `cd <path_to_folder>`
3. Use the requirements.yml file in the root directory to set up the environment: `conda env create -f requirements.yml`
5. Activate the flexus environment: `conda activate flexsus`
5. Open the notebook: `jupyter notebook`

## Description

## Content

## Script set-up

In [1]:
# import packages
import os
import pandas as pd

In [2]:
# make directories
dirs = {'output'}
for i in dirs:
    if not os.path.isdir(i):
        os.mkdir(i)

## Define output resolution

In [138]:
# set of considered countries (if empty, select all)
ccc = {'DENMARK'}
rrr = {'DK1', 'DK2'}

In [4]:
# set of years (if empty, select all)
yyy = {}  # 2025, 2035, 2045

## Load data

In [19]:
# get list of data files
lf = [i.split('.')[0] for i in os.listdir('data')]
#lf = os.listdir('data')

## Load and process data 

In [20]:
def readData(f):
    return pd.read_csv('data/' + f + '.csv', encoding='utf8', engine='c', low_memory=False)

In [41]:
def epsToZeros(df):
    df = df.replace('Eps', 0)
    return df

In [10]:
def filterYearAndCountry(df,y,c):
    if y: df = df.loc[~df.Dim3.isin(set(df.Dim3).difference(y)), :]
    if c: df = df.loc[~df.Dim4.isin(set(df.Dim4).difference(c)), :]
    return df

In [11]:
def makeValFloat(df):
    df.Val = df.Val.astype(float)
    return df

In [38]:
lf

['ECONOMY_ELEC_TRANSMISSION',
 'ECONOMY_GENERATION',
 'ECONOMY_HEAT_TRANSMISSION',
 'ELEC_DEMAND',
 'ELEC_PRICE',
 'ELEC_PRICE_HOURLY',
 'ELEC_TRANSMISSION_CAPACITY',
 'ELEC_TRANSMISSION_FLOW',
 'EMISSIONS_CO2',
 'ENERGY_PRODUCTION',
 'FUEL_CONSUMPTION',
 'GENERATION_CAPACITY',
 'HEAT_DEMAND',
 'HEAT_PRICE',
 'HEAT_PRICE_HOURLY',
 'HEAT_TRANSMISSION_CAPACITY',
 'HEAT_TRANSMISSION_FLOW',
 'STORAGE_CAPACITY',
 'SYSTEM_COSTS']

### ECONOMY_ELEC_TRANSMISSION

In [48]:
lf[0]

'ECONOMY_ELEC_TRANSMISSION'

In [45]:
# ECONOMY_ELEC_TRANSMISSION
df0 = readData(lf[0])
df0 = epsToZeros(df0)
df0 = filterYearAndCountry(df0,yyy,ccc)
df0 = makeValFloat(df0)
df0.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Val
14,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,COSTS,TRANSMISSION_OPERATIONAL_COSTS,Mmoney,0.001645
15,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,COSTS,TRANSMISSION_TRADE_COSTS,Mmoney,34.881906


In [46]:
# group data frame
df0 = df0.groupby(['Dim7','Dim2'])['Val'].sum().unstack().T
df0 = df0.fillna(0)
df0.columns = pd.MultiIndex.from_product([[lf[0]], df0.columns])
df0.head(2)

ECONOMY_ELEC_TRANSMISSION                                 \
Dim7       TRANSMISSION_CAPITAL_COSTS TRANSMISSION_OPERATIONAL_COSTS   
Dim2                                                                   
Scenario1                  417.131961                       0.020614   
Scenario10                 295.729468                       0.009370   

                                                               
Dim7       TRANSMISSION_TRADE_COSTS TRANSMISSION_TRADE_INCOME  
Dim2                                                           
Scenario1                223.822992              10198.187747  
Scenario10               392.203384               5148.391004

### ECONOMY_GENERATION

In [49]:
lf[1]

'ECONOMY_GENERATION'

In [47]:
# ECONOMY_GENERATION
df1 = readData(lf[1])
df1 = epsToZeros(df1)
df1 = filterYearAndCountry(df1,yyy,ccc)
df1 = makeValFloat(df1)
df1.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Dim9,Dim10,Dim11,Dim12,Val
1412,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,GNR_BO_BIOIL_E-85,BIOOIL,BOILERS,COSTS,GENERATION_FIXED_COSTS,Mmoney,0.220143
1413,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,GNR_BO_BIOIL_E-85,BIOOIL,BOILERS,COSTS,GENERATION_OPERATIONAL_COSTS,Mmoney,0.403046


In [50]:
# group data frame
df1 = df1.groupby(['Dim11','Dim2'])['Val'].sum().unstack().T
df1 = df1.fillna(0)
df1.columns = pd.MultiIndex.from_product([[lf[1]], df1.columns])
df1.head(2)

ECONOMY_GENERATION                          \
Dim11        ELECTRICITY_SALE ENERGY_SPECIFIC_REVENUE   
Dim2                                                    
Scenario1        12210.059270            31621.444182   
Scenario10        4098.555644            29221.484573   

                                                                               \
Dim11      GENERATION_CAPITAL_COSTS GENERATION_CO2_TAX GENERATION_FIXED_COSTS   
Dim2                                                                            
Scenario1               6940.383171          86.642451            2664.404139   
Scenario10              3209.519551         219.152990            1049.003067   

                                                          \
Dim11      GENERATION_FUEL_COSTS GENERATION_GRID_TARIFFS   
Dim2                                                       
Scenario1             2620.16481               56.057248   
Scenario10             936.82982              245.660960   

                                                                  \
Dim11      GENERATION_OPERATIONAL_COSTS GENERATION_OTHER_EMI_TAX   
Dim2                                                               
Scenario1                    749.952190                 0.020144   
Scenario10                   538.911484                 0.012272   

                                                                            
Dim11      GENERATION_TAXES GENERATION_UC_COSTS    HEAT_SALE TOTAL_REVENUE  
Dim2                                                                        
Scenario1         87.905015           86.366213  5405.295318   4323.459207  
Scenario10      1971.637897          103.645321  7820.944328   3645.126609

### ECONOMY_HEAT_TRANSMISSION

In [51]:
lf[2]

'ECONOMY_HEAT_TRANSMISSION'

In [52]:
# ECONOMY_HEAT_TRANSMISSION
df2 = readData(lf[2])
df2 = eps2zeros(df2)
df2 = filterYearAndCountry(df2,yyy,ccc)
df2 = makeValFloat(df2)
df2.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Dim9,Val
33,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,COSTS,TRANSMISSION_TRADE_COSTS,Mmoney,24.516919
34,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,COSTS,HEAT_TRANSMISSION_OPERATIONAL_COSTS,Mmoney,0.000001


In [53]:
# group data frame
df2 = df2.groupby(['Dim8','Dim2'])['Val'].sum().unstack().T
df2 = df2.fillna(0)
df2.columns = pd.MultiIndex.from_product([[lf[2]], df2.columns])
df2.head(2)

ECONOMY_HEAT_TRANSMISSION  \
Dim8       HEAT_TRANSMISSION_CAPITAL_COSTS   
Dim2                                         
Scenario1                              0.0   
Scenario10                             0.0   

                                                                         \
Dim8       HEAT_TRANSMISSION_OPERATIONAL_COSTS TRANSMISSION_TRADE_COSTS   
Dim2                                                                      
Scenario1                             0.000225               630.736822   
Scenario10                            0.000518               461.310127   

                                      
Dim8       TRANSMISSION_TRADE_INCOME  
Dim2                                  
Scenario1                 630.736822  
Scenario10                461.310127

## ELEC_DEMAND

In [39]:
lf[3]

'ELEC_DEMAND'

In [62]:
# ELEC_DEMAND
df3 = readData(lf[3])
df3 = epsToZeros(df3)
df3 = filterYearAndCountry(df3,yyy,ccc)
df3 = makeValFloat(df3)
df3.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Val
15,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,EXOGENOUS,TWh,19.791304
16,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,ENDO_EV,TWh,0.589469


In [63]:
# group data frame
df3 = df3.groupby(['Dim6','Dim2'])['Val'].sum().unstack().T
df3 = df3.fillna(0)
df3.columns = pd.MultiIndex.from_product([[lf[3]], df3.columns])
df3.head(2)

ELEC_DEMAND                                                  
Dim6       ENDO_ELBOILER   ENDO_EV ENDO_HEATPUMP ENDO_INTRASTO   EXOGENOUS
Dim2                                                                      
Scenario1       0.017731  7.389184      2.440897      0.549655  100.122479
Scenario10      0.241952  7.389184     19.760165      0.000000  100.122479

## ELEC_PRICE

In [64]:
lf[4]

'ELEC_PRICE'

In [68]:
# ELEC_DEMAND
df4 = readData(lf[4])
df4 = epsToZeros(df4)
df4 = filterYearAndCountry(df4,yyy,ccc)
df4 = makeValFloat(df4)
df4.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Val
5,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,AVERAGE,Money_per_MWh,54.854580
6,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK2,AVERAGE,Money_per_MWh,54.078051


In [69]:
# group data frame
df4 = df4.groupby(['Dim7','Dim2'])['Val'].mean().unstack().T
df4 = df4.fillna(0)
df4.columns = pd.MultiIndex.from_product([[lf[4]], df4.columns])
df4.head(2)

,ELEC_PRICE
Dim7,Money_per_MWh
Dim2,
Scenario1,54.064789
Scenario10,58.679853


## ELEC_PRICE_HOURLY

In [70]:
lf[5]

'ELEC_PRICE_HOURLY'

In [228]:
# ELEC_DEMAND
df5 = readData(lf[5])
df5 = epsToZeros(df5)
df5 = filterYearAndCountry(df5,yyy,ccc)
df5 = makeValFloat(df5)
df5.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Val
625,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,S01,T073,Money_per_MWh,36.322361
626,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,S01,T076,Money_per_MWh,29.869310


In [229]:
# group data frame
df5 = df5.groupby(['Dim6','Dim2'])['Val'].mean().unstack().T
df5 = df5.fillna(0)
df5.columns = pd.MultiIndex.from_product([[lf[5]], df5.columns])
df5.head(2)

ELEC_PRICE_HOURLY                                            
Dim6                     S01        S11        S22        S32        S43
Dim2                                                                    
Scenario1          31.226674  74.843637  46.551809  46.288509  71.413314
Scenario10         34.390692  78.171579  51.718559  54.248490  74.869942

## ELEC_TRANSMISSION_CAPACITY

In [78]:
lf[6]

'ELEC_TRANSMISSION_CAPACITY'

In [233]:
# ELEC_DEMAND
df6 = readData(lf[6])
df6 = epsToZeros(df6)
df6 = filterYearAndCountry(df6,yyy,ccc)
df6 = makeValFloat(df6)
df6['direction'] = df6.Dim5 + '_' + df6.Dim6
df6 = df6.loc[(df6.direction.str.contains('|'.join(rrr))==True), :]
df6.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Val,direction
21,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,NO2,EXOGENOUS,GW,1.632,DK1_NO2
22,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK2,EXOGENOUS,GW,0.590,DK1_DK2


In [234]:
# group data frame
df6 = df6.groupby(['direction','Dim2'])['Val'].sum().unstack().T
df6 = df6.fillna(0)
df6.columns = pd.MultiIndex.from_product([[lf[6]], df6.columns])
df6.head(2)

ELEC_TRANSMISSION_CAPACITY                                         \
direction                   DK1_DE4-N DK1_DK2    DK1_NL   DK1_NO2    DK1_SE3   
Dim2                                                                           
Scenario1                   10.411282    1.77  9.375176  4.896000   9.519111   
Scenario10                   9.060000    1.77  4.089523  5.124768  11.571565   

                                               
direction  DK1_UK DK2_DE4-E DK2_DK1   DK2_SE4  
Dim2                                           
Scenario1     4.2  5.651687     1.8  5.100000  
Scenario10    4.2  2.955000     1.8  6.892446

## ELEC_TRANSMISSION_FLOW

In [83]:
lf[7]

'ELEC_TRANSMISSION_FLOW'

In [180]:
# ELEC_DEMAND
df7 = readData(lf[7])
df7 = epsToZeros(df7)
df7 = filterYearAndCountry(df7,yyy,{})
df7 = makeValFloat(df7)
df7['direction'] = df7.Dim5 + '_' + df7.Dim6
df7 = df7.loc[(df7.direction.str.contains('|'.join(rrr))==True), :]
df7.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Val,direction
5,ScenarioResults_181_270,Scenario181,2025,NORWAY,NO2,DK1,TWh,7.602892,NO2_DK1
19,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,NO2,TWh,1.352829,DK1_NO2


In [181]:
# group data frame
df7 = df7.groupby(['direction','Dim2'])['Val'].sum().unstack().T
df7 = df7.fillna(0)
df7.columns = pd.MultiIndex.from_product([[lf[7]], df7.columns])
df7.head(2)

ELEC_TRANSMISSION_FLOW                                             \
direction               DE4-E_DK2  DE4-N_DK1  DK1_DE4-N   DK1_DK2     DK1_NL   
Dim2                                                                           
Scenario1                0.060723   0.498527  52.263303  3.300211  51.288917   
Scenario10               6.642211  11.211683  22.533587  5.471912  15.292184   

                                                                           \
direction    DK1_NO2    DK1_SE3     DK1_UK  DK2_DE4-E   DK2_DK1   DK2_SE4   
Dim2                                                                        
Scenario1   7.427024  26.622734  18.645123  30.606655  1.998565  9.547093   
Scenario10  2.452366  22.249023  13.229752   8.620229  1.330232  0.499585   

                                                                  
direction     NL_DK1    NO2_DK1    SE3_DK1    SE4_DK2     UK_DK1  
Dim2                                                              
Scenario1   4.493234  13.833772  31.765463  10.255517   6.740544  
Scenario10  6.950268  26.196357  49.469099  28.458313  10.171865

## EMISSIONS_CO2

In [135]:
lf[8]

'EMISSIONS_CO2'

In [193]:
# EMISSIONS_CO2
df8 = readData(lf[8])
df8 = epsToZeros(df8)
df8 = filterYearAndCountry(df8,yyy,ccc)
df8 = makeValFloat(df8)
df8['technology'] = df8.Dim8 + '_' + df8.Dim9
df8.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Dim9,Dim10,Val,technology
35,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,GNR_BO_NGAS_E-104,NATGAS,BOILERS,kton,1.616068,NATGAS_BOILERS
36,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,GNR_ENG_LIGHTOIL_BP_E-37,LIGHTOIL,CHP-BACK-PRESSURE,kton,2.950632,LIGHTOIL_CHP-BACK-PRESSURE


In [194]:
# group data frame
df8 = df8.groupby(['technology','Dim2'])['Val'].sum().unstack().T
df8 = df8.fillna(0)
df8.columns = pd.MultiIndex.from_product([[lf[8]], df8.columns])
df8.head(2)

EMISSIONS_CO2                                             \
technology  COAL_BOILERS COAL_CHP-BACK-PRESSURE COAL_CHP-EXTRACTION   
Dim2                                                                  
Scenario1       0.000000              41.917468        6.883196e-09   
Scenario10      5.326626             894.718410        4.328434e+03   

                                                                      \
technology COAL_CONDENSING FUELOIL_BOILERS FUELOIL_CHP-BACK-PRESSURE   
Dim2                                                                   
Scenario1              0.0        0.000000                  0.000000   
Scenario10             0.0        0.439693                  1.958605   

                                                                           \
technology FUELOIL_CONDENSING LIGHTOIL_BOILERS LIGHTOIL_CHP-BACK-PRESSURE   
Dim2                                                                        
Scenario1            0.031298         0.000000                   6.170311   
Scenario10           0.031298         5.551103                   6.854302   

                                                                        \
technology LIGHTOIL_CONDENSING NATGAS_BOILERS NATGAS_CHP-BACK-PRESSURE   
Dim2                                                                     
Scenario1             9.072628       2.858547                55.022099   
Scenario10            9.072628     897.469777               211.306336   

                                                    
technology NATGAS_CHP-EXTRACTION NATGAS_CONDENSING  
Dim2                                                
Scenario1                    0.0        773.219416  
Scenario10                   0.0          6.199197

## ENERGY_PRODUCTION

In [187]:
lf[9]

'ENERGY_PRODUCTION'

In [190]:
# ENERGY_PRODUCTION
df9 = readData(lf[9])
df9 = epsToZeros(df9)
df9 = filterYearAndCountry(df9,yyy,ccc)
df9 = makeValFloat(df9)
df9['technology'] = df9.Dim9 + '_' + df9.Dim8 + '_' + df9.Dim10
df9.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Dim9,Dim10,Dim11,Val,technology
214,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,GNR_BO_BIOIL_E-85,BIOOIL,HEAT,BOILERS,TWh,0.365835,HEAT_BIOOIL_BOILERS
215,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,GNR_BO_BIOIL_E-93,BIOOIL,HEAT,BOILERS,TWh,0.077228,HEAT_BIOOIL_BOILERS


In [191]:
# group data frame
df9 = df9.groupby(['technology','Dim2'])['Val'].sum().unstack().T
df9 = df9.fillna(0)
df9.columns = pd.MultiIndex.from_product([[lf[9]], df9.columns])
df9.head(2)

ENERGY_PRODUCTION                                \
technology ELECTRICITY_BIOGAS_CHP-BACK-PRESSURE ELECTRICITY_BIOGAS_CONDENSING   
Dim2                                                                            
Scenario1                              0.142967                      0.000373   
Scenario10                             1.977862                      0.000250   

                                                                               \
technology ELECTRICITY_COAL_CHP-BACK-PRESSURE ELECTRICITY_COAL_CHP-EXTRACTION   
Dim2                                                                            
Scenario1                            0.046772                    6.815530e-12   
Scenario10                           0.979251                    4.641724e+00   

                                        \
technology ELECTRICITY_COAL_CONDENSING   
Dim2                                     
Scenario1                          0.0   
Scenario10                         0.0   

                                                             \
technology ELECTRICITY_ELECTRIC_INTRASEASONAL-ELECT-STORAGE   
Dim2                                                          
Scenario1                                          7.647365   
Scenario10                                         0.000000   

                                                  \
technology ELECTRICITY_FUELOIL_CHP-BACK-PRESSURE   
Dim2                                               
Scenario1                                 0.0000   
Scenario10                                0.0025   

                                           \
technology ELECTRICITY_FUELOIL_CONDENSING   
Dim2                                        
Scenario1                        0.000049   
Scenario10                       0.000049   

                                                   \
technology ELECTRICITY_LIGHTOIL_CHP-BACK-PRESSURE   
Dim2                                                
Scenario1                                0.008555   
Scenario10                               0.009506   

                                            ...                     \
technology ELECTRICITY_LIGHTOIL_CONDENSING  ... HEAT_STRAW_BOILERS   
Dim2                                        ...                      
Scenario1                         0.013303  ...           4.020378   
Scenario10                        0.013303  ...           0.147762   

                                                                \
technology HEAT_STRAW_CHP-BACK-PRESSURE HEAT_SUN_SOLAR-HEATING   
Dim2                                                             
Scenario1                     15.861445               6.048965   
Scenario10                     0.750449               1.145687   

                                                          \
technology HEAT_WASTEHEAT_BOILERS HEAT_WOODCHIPS_BOILERS   
Dim2                                                       
Scenario1                0.474354               5.069544   
Scenario10               0.474354              31.960424   

                                                                      \
technology HEAT_WOODCHIPS_CHP-BACK-PRESSURE HEAT_WOODPELLETS_BOILERS   
Dim2                                                                   
Scenario1                         83.567709                  0.00000   
Scenario10                         4.428785                  0.09084   

                                                                               \
technology HEAT_WOODPELLETS_CHP-BACK-PRESSURE HEAT_WOODPELLETS_CHP-EXTRACTION   
Dim2                                                                            
Scenario1                            2.631275                        1.066647   
Scenario10                           0.007574                        1.064476   

                                   
technology HEAT_WOODWASTE_BOILERS  
Dim2                               
Scenario1                0.831019  
Scenario10               0.799486  

[2 rows

## FUEL_CONSUMPTION

In [195]:
lf[10]

'FUEL_CONSUMPTION'

In [200]:
# FUEL_CONSUMPTION
df10 = readData(lf[10])
df10 = epsToZeros(df10)
df10 = filterYearAndCountry(df10,yyy,ccc)
df10 = makeValFloat(df10)
df10['technology'] = df10.Dim8 + '_' + df10.Dim9
df10.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Dim9,Dim10,Val,technology
196,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,GNR_BO_BIOIL_E-85,BIOOIL,BOILERS,TWh,0.430394,BIOOIL_BOILERS
197,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,GNR_BO_BIOIL_E-93,BIOOIL,BOILERS,TWh,0.083041,BIOOIL_BOILERS


In [201]:
# group data frame
df10 = df10.groupby(['Dim8','Dim2'])['Val'].sum().unstack().T
df10 = df10.fillna(0)
df10.columns = pd.MultiIndex.from_product([[lf[10]], df10.columns])
df10.head(2)

FUEL_CONSUMPTION                                           \
Dim8                 BIOGAS   BIOOIL       COAL   ELECTRIC   FUELOIL   
Dim2                                                                   
Scenario1          0.335388  0.00000   0.123084  10.655648  0.000117   
Scenario10         4.426154  1.38773  15.352594  20.002117  0.009120   

                                                                            \
Dim8             HEAT  LIGHTOIL  MUNIWASTE    NATGAS      STRAW        SUN   
Dim2                                                                         
Scenario1   27.200714  0.057218  32.617654  4.115172  22.098829  48.799530   
Scenario10   0.000000  0.080623  31.989164  5.520773   1.164413   2.847284   

                                                                             
Dim8       WASTEHEAT     WATER        WIND  WOODCHIPS WOODPELLETS WOODWASTE  
Dim2                                                                         
Scenario1   0.474354  0.100566  163.564812  95.554043    6.035170  0.870664  
Scenario10  0.474354  0.100566   59.727893  32.983738    2.020647  0.836717

## GENERATION_CAPACITY

In [202]:
lf[11]

'GENERATION_CAPACITY'

In [203]:
# GENERATION_CAPACITY
df11 = readData(lf[11])
df11 = epsToZeros(df11)
df11 = filterYearAndCountry(df11,yyy,ccc)
df11 = makeValFloat(df11)
df11['technology'] = df11.Dim8 + '_' + df11.Dim9
df11.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Dim9,Dim10,Dim11,Dim12,Val,technology
432,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,GNR_BO_ELEC_E-100,ELECTRIC,HEAT,ELECT-TO-HEAT,EXOGENOUS,GW,0.12895,ELECTRIC_HEAT
433,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,GNR_BO_ELEC_E-100,ELECTRIC,HEAT,ELECT-TO-HEAT,DECOMMISSIONING,GW,-0.12895,ELECTRIC_HEAT


In [211]:
# group data frame
df11 = df11.groupby(['Dim10','Dim2'])['Val'].sum().unstack().T
df11 = df11.fillna(0)
df11.columns = pd.MultiIndex.from_product([[lf[11]], df11.columns])
df11.head(2)

GENERATION_CAPACITY                                              \
Dim10                  BOILERS CHP-BACK-PRESSURE CHP-EXTRACTION CONDENSING   
Dim2                                                                         
Scenario1             1.690526         23.202771       0.588961  15.099901   
Scenario10           17.240726          9.776967       2.508599   0.149460   

                                                                        \
Dim10      ELECT-TO-HEAT HYDRO-RUN-OF-RIVER INTERSEASONAL-HEAT-STORAGE   
Dim2                                                                     
Scenario1       2.344929           0.020865                   6.921221   
Scenario10     14.107220           0.020865                   0.000000   

                                                                   \
Dim10      INTRASEASONAL-ELECT-STORAGE INTRASEASONAL-HEAT-STORAGE   
Dim2                                                                
Scenario1                     5.351014                  14.620806   
Scenario10                    0.000000                   0.000000   

                                                           
Dim10      SOLAR-HEATING   SOLAR-PV   WIND-OFF    WIND-ON  
Dim2                                                       
Scenario1       8.640058  41.818506  22.559131  24.000000  
Scenario10      1.648190   1.747898   5.591300  13.210993

## HEAT_DEMAND

In [213]:
lf[12]

'HEAT_DEMAND'

In [217]:
# HEAT_DEMAND
df12 = readData(lf[12])
df12 = epsToZeros(df12)
df12 = filterYearAndCountry(df12,yyy,ccc)
df12 = makeValFloat(df12)
df12.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Val
45,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,EXOGENOUS,TWh,8.140747
46,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,ENDO_INTRASTO,TWh,0.004883


In [218]:
# group data frame
df12 = df12.groupby(['Dim7','Dim2'])['Val'].sum().unstack().T
df12 = df12.fillna(0)
df12.columns = pd.MultiIndex.from_product([[lf[12]], df12.columns])
df12.head(2)

HEAT_DEMAND                          
Dim7       ENDO_INTERSTO ENDO_INTRASTO   EXOGENOUS
Dim2                                              
Scenario1       4.652332      0.298073  149.765004
Scenario10      0.000000      0.000000  149.765004

## HEAT_PRICE

In [219]:
lf[13]

'HEAT_PRICE'

In [224]:
# HEAT_PRICE
df13 = readData(lf[13])
df13 = epsToZeros(df13)
df13 = filterYearAndCountry(df13,yyy,ccc)
df13 = makeValFloat(df13)
df13.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Val
27,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,AVERAGE,Money_per_MWh,49.833987
28,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Rural,AVERAGE,Money_per_MWh,46.933958


In [225]:
# group data frame
df13 = df13.groupby(['Dim7','Dim2'])['Val'].mean().unstack().T
df13 = df13.fillna(0)
df13.columns = pd.MultiIndex.from_product([[lf[13]], df13.columns])
df13.head(2)

,HEAT_PRICE
Dim7,AVERAGE
Dim2,
Scenario1,29.245563
Scenario10,43.473431


## HEAT_PRICE_HOURLY

In [226]:
lf[14]

'HEAT_PRICE_HOURLY'

In [230]:
# EMISSIONS_CO2
df14 = readData(lf[14])
df14 = epsToZeros(df14)
df14 = filterYearAndCountry(df14,yyy,ccc)
df14 = makeValFloat(df14)
df14.head(2)

ParserError: Error tokenizing data. C error: out of memory

In [225]:
# group data frame
df14 = df14.groupby(['Dim7','Dim2'])['Val'].mean().unstack().T
df14 = df14.fillna(0)
df14.columns = pd.MultiIndex.from_product([[lf[14]], df14.columns])
df14.head(2)

,HEAT_PRICE
Dim7,AVERAGE
Dim2,
Scenario1,29.245563
Scenario10,43.473431


## HEAT_TRANSMISSION_CAPACITY

In [231]:
lf[15]

'HEAT_TRANSMISSION_CAPACITY'

In [241]:
# EMISSIONS_CO2
df15 = readData(lf[15])
df15 = epsToZeros(df15)
df15 = filterYearAndCountry(df15,yyy,ccc)
df15 = makeValFloat(df15)
df15['direction'] = df15.Dim5.str.split('_').str[0] + '_' + df15.Dim6.str.split('_').str[0]
df15.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Val,direction
20,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1_Large,DK1_IND-LT-DH,EXOGENOUS,GW,0.257,DK1_DK1
21,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1_IND-HT-NODH,DK1_IND-LT-NODH,EXOGENOUS,GW,1000.000,DK1_DK1


In [242]:
# group data frame
df15 = df15.groupby(['direction','Dim2'])['Val'].sum().unstack().T
df15 = df15.fillna(0)
df15.columns = pd.MultiIndex.from_product([[lf[15]], df15.columns])
df15.head(2)

HEAT_TRANSMISSION_CAPACITY          
direction                     DK1_DK1   DK2_DK2
Dim2                                           
Scenario1                    6001.542  6001.056
Scenario10                   6001.542  6001.056

## HEAT_TRANSMISSION_FLOW

In [243]:
lf[16]

'HEAT_TRANSMISSION_FLOW'

In [251]:
# EMISSIONS_CO2
df16 = readData(lf[16])
df16 = epsToZeros(df16)
df16 = filterYearAndCountry(df16,yyy,{})
df16 = makeValFloat(df16)
df16['direction'] = df16.Dim5.str.split('_').str[0] + '_' + df16.Dim6.str.split('_').str[0]
df16['direction'] = df16.Dim5 + '_' + df16.Dim6
df16 = df16.loc[(df16.direction.str.contains('|'.join(rrr))==True), :]
df16.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Val,direction
16,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1_Large,DK1_IND-LT-DH,TWh,0.010155,DK1_Large_DK1_IND-LT-DH
17,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1_IND-HT-NODH,DK1_IND-LT-NODH,TWh,2.206051,DK1_IND-HT-NODH_DK1_IND-LT-NODH


In [252]:
# group data frame
df16 = df16.groupby(['Dim6','Dim2'])['Val'].sum().unstack().T
df16 = df16.fillna(0)
df16.columns = pd.MultiIndex.from_product([[lf[16]], df16.columns])
df16.head(2)

HEAT_TRANSMISSION_FLOW                                          \
Dim6                DK1_IND-LT-DH DK1_IND-LT-NODH DK1_Large DK2_IND-LT-DH   
Dim2                                                                        
Scenario1                2.902567        8.551676  0.380031      1.922255   
Scenario10               2.551515        1.588260  2.254063      1.631079   

                                      
Dim6       DK2_IND-LT-NODH DK2_Large  
Dim2                                  
Scenario1         5.848884  0.181006  
Scenario10        1.355724  1.350027

## STORAGE_CAPACITY

In [253]:
lf[17]

'STORAGE_CAPACITY'

In [260]:
# EMISSIONS_CO2
df17 = readData(lf[17])
df17 = epsToZeros(df17)
df17 = filterYearAndCountry(df17,yyy,ccc)
df17 = makeValFloat(df17)
df17.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Dim8,Dim9,Dim10,Dim11,Dim12,Val
18,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,GNR_HS_HEAT_PIT_L-CEN_E-70_Y-2020,HEAT,HEAT,INTERSEASONAL-HEAT-STORAGE,ENDOGENOUS,GWh,34.896348
19,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,DK1_Large,GNR_HS_HEAT_WTR-TANK_SS_E-95_Y-2010,HEAT,HEAT,INTRASEASONAL-HEAT-STORAGE,EXOGENOUS,GWh,6.730400


In [261]:
# group data frame
df17 = df17.groupby(['Dim10','Dim2'])['Val'].sum().unstack().T
df17 = df17.fillna(0)
df17.columns = pd.MultiIndex.from_product([[lf[17]], df17.columns])
df17.head(2)

STORAGE_CAPACITY                              \
Dim10      INTERSEASONAL-HEAT-STORAGE INTRASEASONAL-ELECT-STORAGE   
Dim2                                                                
Scenario1                  799.197444                   21.404056   
Scenario10                   0.000000                    0.000000   

                                       
Dim10      INTRASEASONAL-HEAT-STORAGE  
Dim2                                   
Scenario1                  331.346904  
Scenario10                   0.000000

## SYSTEM_COSTS

In [262]:
lf[18]

'SYSTEM_COSTS'

In [263]:
# EMISSIONS_CO2
df18 = readData(lf[18])
df18 = epsToZeros(df18)
df18 = filterYearAndCountry(df18,yyy,ccc)
df18 = makeValFloat(df18)
df18.head(2)

,Dim1,Dim2,Dim3,Dim4,Dim5,Dim6,Dim7,Val
60,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,GENERATION_FIXED_COSTS,Mmoney,268.345965
61,ScenarioResults_181_270,Scenario181,2025,DENMARK,DK1,GENERATION_OPERATIONAL_COSTS,Mmoney,110.026918


In [264]:
# group data frame
df18 = df18.groupby(['Dim6','Dim2'])['Val'].sum().unstack().T
df18 = df18.fillna(0)
df18.columns = pd.MultiIndex.from_product([[lf[18]], df18.columns])
df18.head(2)

SYSTEM_COSTS                                            \
Dim6       GENERATION_CAPITAL_COSTS GENERATION_CO2_TAX GENERATION_FIXED_COSTS   
Dim2                                                                            
Scenario1               6940.383171          86.642451            2664.404139   
Scenario10              3209.519551         219.152990            1049.003067   

                                                               \
Dim6       GENERATION_FUEL_COSTS GENERATION_OPERATIONAL_COSTS   
Dim2                                                            
Scenario1            2606.018031                   749.952190   
Scenario10            922.158465                   538.911484   

                                                                      \
Dim6       GENERATION_OTHER_EMI_TAX GENERATION_UC_COSTS GRID_TARIFFS   
Dim2                                                                   
Scenario1                  0.020144           86.366213  4317.979857   
Scenario10                 0.012272          103.645321  4507.583569   

                                            \
Dim6       HEAT_TRANSMISSION_CAPITAL_COSTS   
Dim2                                         
Scenario1                              0.0   
Scenario10                             0.0   

                                                              \
Dim6       HEAT_TRANSMISSION_OPERATIONAL_COSTS         TAXES   
Dim2                                                           
Scenario1                             0.000225   8188.898895   
Scenario10                            0.000518  10072.631777   

                                                                      
Dim6       TRANSMISSION_CAPITAL_COSTS TRANSMISSION_OPERATIONAL_COSTS  
Dim2                                                                  
Scenario1                  417.131961                       0.020614  
Scenario10                 295.729468                       0.009370

TypeError: eval() arg 1 must be a string, bytes or code object

In [286]:
frames = [eval('df' + str(i)) for i in range(len(lf))]
frames

NameError: name 'df14' is not defined

In [278]:
frames = ['df' + str(i) for i in range(len(lf))]
pd.concat([df1, df4], axis=1, sort=False)

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid